In [ ]:
import tensorflow as tf
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.9.2


In [ ]:
msg = tf.constant('Hello Tensorflow!')
tf.print(msg)

Hello Tensorflow!


In [ ]:
import keras
print('Keras Version:', keras.__version__)

Keras Version: 2.9.0


In [ ]:
from keras import datasets
from keras import models
from keras import layers
from keras.utils import to_categorical

# Load MNIST data
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Creating network consists of two dense layers(fully connected layer)
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,), name="layer1"))
network.add(layers.Dense(10, activation='softmax', name="layer2"))

# Preparing the training images and labels
train_images = train_images.reshape(60000, 28*28)
train_images = train_images.astype('float32') / 255
train_labels = to_categorical(train_labels)

# Preparing the testing images and labels
test_images = test_images.reshape(10000, 28*28)
test_images = test_images.astype('float32') / 255
test_labels = to_categorical(test_labels)

# Prepare the network --> rmsprop : root mean square gradient descent
network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the neural network
network.fit(train_images, train_labels, epochs=5, batch_size=128)

# Evaluate the network performance
test_loss, test_acc = network.evaluate(test_images, test_labels)

# Print testing accuracy and testing loss
print('Testing : ', test_acc)
print('Training : ', test_loss)

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.2545 - accuracy: 0.9252
Epoch 2/5
469/469 [==============================] - 6s 13ms/step - loss: 0.1036 - accuracy: 0.9690
Epoch 3/5
469/469 [==============================] - 7s 14ms/step - loss: 0.0689 - accuracy: 0.9790
Epoch 4/5
469/469 [==============================] - 6s 13ms/step - loss: 0.0501 - accuracy: 0.9856
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0675 - accuracy: 0.9788
Testing :  0.9787999987602234
Training :  0.06750303506851196
